In [2]:
import os
import json
import pandas as pd
import numpy as np
from nltk.translate import bleu_score

os.chdir(r"D:\박준호\2024년\NLP-Standard-Dialect-Transformation")
pd.set_option('display.max_colwidth', None)

In [4]:
from metric import compute_chrf_score

강원도 방언 test set 개수 맞추기

In [ ]:
r = "gangwon"
m = "opus-mt"
path_to_trans_result = "./translation_english/prediction/"
path_to_reference = "./translation_english/reference/"

trans_file_path = os.path.join(path_to_trans_result, "{0}/{1}.json".format(r, m))
with open(trans_file_path, mode="rt", encoding="utf-8") as f:
    json_list = json.loads(f.read())
result_df = pd.DataFrame(json_list)

ref_file_path = os.path.join(path_to_reference, "{0}/reference.json".format(r))
with open(ref_file_path, mode="rt", encoding="utf-8") as f:
    json_list = json.loads(f.read())
ref_df = pd.DataFrame(json_list)

# 왜 두 개 길이가 다른가?
opus_source_list = result_df["prediction_source"].to_list()
opus_source_dict = dict(zip(opus_source_list, range(len(opus_source_list))))

excluded_list = []
for i in range(len(ref_df)):
    row = ref_df.iloc[i]
    if row["standard_source"] not in opus_source_dict:
        excluded_list.append(i)

In [23]:
ref_df.iloc[excluded_list]

,standard_source,standard_target
91328,어~ 음식 중에 이제 음식 중에 하나인 것 같아 그리고,I think it's one of the foods now.
91329,음~ 어~ 이제 뼛국 같은 경우도 지금은 해 주 어~ 없어서 못 먹지.,"I think it would be better to translate the sentence as a whole rather than breaking it down. Here is the translation:\n\n""Ah, um, I guess I won't be able to eat it now, huh?"""
91330,어~ 없어서 못 먹지 그게 굉장한 오랜 시간의 정성이 필요한 음식이잖아.,"That's a tough one to give up, you know it takes a lot of effort and time to make that food."
91331,엄청 큰 솥단지에다가 이제 뼈랑 이게 국,A gigantic pot with bones and this is a soup.
91332,그걸 으 야채라든가 이런 걸 넣고 푹 이제,That stuff in it or add some vegetables and now it's okay.
91333,끓이면서 위에 있는 불순물 떠 내고,Boiling and then removing the impurities that float on top.
91334,굉장히 약불로 하다가 굉장한 오랜 시간 동안,Very faintly for a very long time.
91335,그래서 보면은 항상 어머니가 음~ 그걸 만들 때는 큰 솥단지에다가 항상 끓이셨는데,"So then I see, it's always my mom who does it. When she makes that, she always boils it in a big pot."
91336,그러면은 그걸 가지고 진짜 한 한 달은 먹었었던 것 같아 진짜,I think I really ate that for a whole month.
91337,내가 그래서 싫어했었던 것도 같아 왜냐면 한 번에 할 때 많은 양을 하다 보니까,"I used to dislike that too because when I do it all at once, I end up doing a lot."


이거 그냥 길이로 잘라 버렸네..

대체 왜 그런 짓을?

## 미중일 dirct 성능

In [11]:
def chrf_of_direct_translation(path_to_trans_result, path_to_reference, regions, models, word_order=2):
    
    eval_dict = dict()
    for r in regions:
        print(r)
        result_dict = dict()
        df_len = 0
        for m in models:
            json_list = []
            trans_file_path = os.path.join(path_to_trans_result, "{0}/{1}.json".format(r, m))
            with open(trans_file_path, mode="rt", encoding="utf-8") as f:
                json_list = json.loads(f.read())
            
            df = pd.DataFrame(json_list)
            result_dict[m] = df
            if df_len > 0:
                df_len = min(df_len, len(df))
            else:
                df_len = len(df)
        
        # Load reference data
        ref_file_path = os.path.join(path_to_reference, "{0}/reference.json".format(r))
        with open(ref_file_path, mode="rt", encoding="utf-8") as f:
            json_list = json.loads(f.read())
        
        ref_df = pd.DataFrame(json_list)
        
        r_eval_dict = dict()
        
        # Ensure the reference DataFrame has the same length as the result DataFrames
        
        
        for m in models:
            len_result = len(result_dict[m])
            r_eval_dict[m] = compute_chrf_score(
                y_true=ref_df.iloc[range(len_result)]["standard_target"].to_list(), 
                y_pred=result_dict[m]["dialect_target"].to_list(),
                word_order=word_order
            )

        eval_dict[r] = r_eval_dict
    
    return eval_dict

In [ ]:
regions = ["chungcheong", "gangwon", "gyeongsang", "jeju", "jeonla"]
models = ["opus-mt", "m2m_100_1.2B", "exaone"]

eval_dict_chrf = chrf_of_direct_translation("./translation_en/baseline/", "./translation_en/reference/", regions, models)

chungcheong
gangwon
gyeongsang
jeju
jeonla


In [8]:
print(eval_dict_chrf)

{'chungcheong': {'opus-mt': 29.899736299710117, 'm2m_100_1.2B': 29.927620701656622, 'exaone': 41.84524814235252}, 'gangwon': {'opus-mt': 28.11569028009181, 'm2m_100_1.2B': 27.828509422928153, 'exaone': 39.58344502296015}, 'gyeongsang': {'opus-mt': 29.890192821744694, 'm2m_100_1.2B': 29.688938767097884, 'exaone': 42.18258264894414}, 'jeju': {'opus-mt': 25.458562069697656, 'm2m_100_1.2B': 25.157719904722757, 'exaone': 36.639162061946465}, 'jeonla': {'opus-mt': 28.790851705080385, 'm2m_100_1.2B': 28.846113804400535, 'exaone': 40.73570025731394}}


In [33]:
regions = ["chungcheong", "gangwon", "gyeongsang", "jeju", "jeonla"]
models = ["m2m_100_1.2B", "exaone"]

eval_dict_chrf = chrf_of_direct_translation("./translation_zh/baseline/", "./translation_zh/reference/", regions, models, word_order=0)

chungcheong
gangwon
gyeongsang
jeju
jeonla


In [34]:
print(eval_dict_chrf)

{'chungcheong': {'m2m_100_1.2B': 10.233893096873826, 'exaone': 13.870981488308445}, 'gangwon': {'m2m_100_1.2B': 9.506399012629473, 'exaone': 13.196685395020877}, 'gyeongsang': {'m2m_100_1.2B': 10.572085041798958, 'exaone': 14.480980567018761}, 'jeju': {'m2m_100_1.2B': 8.736314809763911, 'exaone': 12.103708324401264}, 'jeonla': {'m2m_100_1.2B': 9.958133562191037, 'exaone': 13.488134644613927}}


In [15]:
regions = ["chungcheong", "gangwon", "gyeongsang", "jeju", "jeonla"]
models = ["m2m_100_1.2B", "exaone"]

eval_dict_chrf = chrf_of_direct_translation("./translation_jp/baseline/", "./translation_jp/reference/", regions, models, word_order=0)

chungcheong
gangwon
gyeongsang
jeju
jeonla


In [16]:
print(eval_dict_chrf)

{'chungcheong': {'m2m_100_1.2B': 14.257823234391228, 'exaone': 20.964232363478796}, 'gangwon': {'m2m_100_1.2B': 12.520343934959389, 'exaone': 19.435285698119962}, 'gyeongsang': {'m2m_100_1.2B': 14.329880040577816, 'exaone': 21.508564801948456}, 'jeju': {'m2m_100_1.2B': 10.760888324268258, 'exaone': 17.06342426369407}, 'jeonla': {'m2m_100_1.2B': 13.8149272022079, 'exaone': 20.435548142894913}}


In [27]:
region = "chungcheong"
model = "exaone"
path_to_translation = "./translation_english/"

file_path = os.path.join(path_to_translation, "reference/{0}/reference.json".format(region))
    
json_list = []

with open(file_path, mode="rt", encoding="utf-8") as f:
    json_list = json.loads(f.read())

reference_df = pd.DataFrame(json_list)

file_path = os.path.join(path_to_translation, "prediction/{0}/{1}.json".format(region, model))
        
with open(file_path, mode="rt", encoding="utf-8") as f:
    json_list = json.loads(f.read())

pred_df = pd.DataFrame(json_list)

In [28]:
print(len(pred_df), len(reference_df))

95000 95000


In [18]:
def chrf_of_prediction(path_to_translation, region, model, word_order=2):
    file_path = os.path.join(path_to_translation, "reference/{0}/reference.json".format(region))
    
    json_list = []

    with open(file_path, mode="rt", encoding="utf-8") as f:
        json_list = json.loads(f.read())

    reference_df = pd.DataFrame(json_list)
    
    file_path = os.path.join(path_to_translation, "prediction/{0}/{1}.json".format(region, model))
        
    with open(file_path, mode="rt", encoding="utf-8") as f:
        json_list = json.loads(f.read())
        
    pred_df = pd.DataFrame(json_list)
    
    # 방언 부분집합 걸러내기
    
    # 두가지 점수 구하기
    score_dict = dict()
    
    len_result = len(pred_df)
    pred_score = compute_chrf_score(
        y_pred=pred_df["prediction_target"].to_list(),
        y_true=reference_df.iloc[range(len_result)]["standard_target"].to_list(),
        word_order=word_order
    )
    
    score_dict={"pred_chrf": pred_score}
    
    return score_dict

## 영어 pivot 성능

In [19]:
eval_dict = dict()

regions = ["chungcheong", "gangwon", "gyeongsang", "jeju", "jeonla"]

for r in regions:
    print(r)
    score_dict = chrf_of_prediction("./translation_en/", r, "opus-mt")
    eval_dict[r] = score_dict
    
print(eval_dict)

chungcheong
gangwon
gyeongsang
jeju
jeonla
{'chungcheong': {'pred_chrf': 30.041264515912943}, 'gangwon': {'pred_chrf': 28.35038718480748}, 'gyeongsang': {'pred_chrf': 30.06574848038776}, 'jeju': {'pred_chrf': 26.916914658319385}, 'jeonla': {'pred_chrf': 28.869409988253203}}


In [20]:
for r in regions:
    print(r)
    score_dict = chrf_of_prediction("./translation_en/", r, "m2m_100_1.2B")
    eval_dict[r] = score_dict
    
print(eval_dict)

chungcheong
gangwon
gyeongsang
jeju
jeonla
{'chungcheong': {'pred_chrf': 30.14948841492747}, 'gangwon': {'pred_chrf': 28.12401964733348}, 'gyeongsang': {'pred_chrf': 29.841671013957555}, 'jeju': {'pred_chrf': 26.690496367237614}, 'jeonla': {'pred_chrf': 29.0406218919269}}


In [21]:
for r in regions:
    print(r)
    score_dict = chrf_of_prediction("./translation_en/", r, "exaone")
    eval_dict[r] = score_dict
    
print(eval_dict)

chungcheong
gangwon
gyeongsang
jeju
jeonla
{'chungcheong': {'pred_chrf': 41.9058010544727}, 'gangwon': {'pred_chrf': 39.61855267669251}, 'gyeongsang': {'pred_chrf': 42.09081791831114}, 'jeju': {'pred_chrf': 38.13848886796272}, 'jeonla': {'pred_chrf': 40.593375002203274}}


## 일본어 pivot 성능

In [31]:
eval_dict = dict()

regions = ["chungcheong", "gangwon", "gyeongsang", "jeju", "jeonla"]

for r in regions:
    print(r)
    score_dict = chrf_of_prediction("./translation_jp/", r, "m2m_100_1.2B", word_order=0)
    eval_dict[r] = score_dict
    
print(eval_dict)

chungcheong
gangwon
gyeongsang
jeju
jeonla
{'chungcheong': {'pred_chrf': 14.380678718196432}, 'gangwon': {'pred_chrf': 12.68876939091021}, 'gyeongsang': {'pred_chrf': 14.424158191952651}, 'jeju': {'pred_chrf': 11.599434528493884}, 'jeonla': {'pred_chrf': 13.857333367274165}}


In [32]:
eval_dict = dict()

regions = ["chungcheong", "gangwon", "gyeongsang", "jeju", "jeonla"]

for r in regions:
    print(r)
    score_dict = chrf_of_prediction("./translation_jp/", r, "exaone", word_order=0)
    eval_dict[r] = score_dict
    
print(eval_dict)

chungcheong
gangwon
gyeongsang
jeju
jeonla
{'chungcheong': {'pred_chrf': 21.050896101710048}, 'gangwon': {'pred_chrf': 19.503125276915007}, 'gyeongsang': {'pred_chrf': 21.60154817262145}, 'jeju': {'pred_chrf': 17.85871736903309}, 'jeonla': {'pred_chrf': 20.39325998434969}}


## 중국어 pivot 성능

In [35]:
eval_dict = dict()

regions = ["chungcheong", "gangwon", "gyeongsang", "jeju", "jeonla"]

for r in regions:
    print(r)
    score_dict = chrf_of_prediction("./translation_zh/", r, "m2m_100_1.2B", word_order=0)
    eval_dict[r] = score_dict
    
print(eval_dict)

chungcheong
gangwon
gyeongsang
jeju
jeonla
{'chungcheong': {'pred_chrf': 10.334205253657267}, 'gangwon': {'pred_chrf': 9.648737011157108}, 'gyeongsang': {'pred_chrf': 10.62066027505951}, 'jeju': {'pred_chrf': 9.44489134653512}, 'jeonla': {'pred_chrf': 10.008641792506038}}


In [36]:
eval_dict = dict()

regions = ["chungcheong", "gangwon", "gyeongsang", "jeju", "jeonla"]

for r in regions:
    print(r)
    score_dict = chrf_of_prediction("./translation_zh/", r, "exaone", word_order=0)
    eval_dict[r] = score_dict
    
print(eval_dict)

chungcheong
gangwon
gyeongsang
jeju
jeonla
{'chungcheong': {'pred_chrf': 13.896448603001646}, 'gangwon': {'pred_chrf': 13.221778251097671}, 'gyeongsang': {'pred_chrf': 14.433445920854501}, 'jeju': {'pred_chrf': 12.735064472551002}, 'jeonla': {'pred_chrf': 13.423115083350293}}


# 방언 부분집합에서의 chrf 성능

# 방언 부분집합의 비율

In [38]:
r = "gyeongsang"
m = "opus-mt"
path_to_trans_result = "./translation_en/baseline/"
path_to_reference = "./translation_en/reference/"

trans_file_path = os.path.join(path_to_trans_result, "{0}/{1}.json".format(r, m))
with open(trans_file_path, mode="rt", encoding="utf-8") as f:
    json_list = json.loads(f.read())

result_df = pd.DataFrame(json_list)
dialect_df = result_df[result_df["dialect_source"] != result_df["standard_source"]]

In [39]:
print(len(result_df), len(dialect_df))

89472 14174


In [40]:
r = "jeonla"

trans_file_path = os.path.join(path_to_trans_result, "{0}/{1}.json".format(r, m))
with open(trans_file_path, mode="rt", encoding="utf-8") as f:
    json_list = json.loads(f.read())

result_df = pd.DataFrame(json_list)
dialect_df = result_df[result_df["dialect_source"] != result_df["standard_source"]]

In [41]:
print(len(result_df), len(dialect_df))

110080 25727


In [42]:
r = "jeju"

trans_file_path = os.path.join(path_to_trans_result, "{0}/{1}.json".format(r, m))
with open(trans_file_path, mode="rt", encoding="utf-8") as f:
    json_list = json.loads(f.read())

result_df = pd.DataFrame(json_list)
dialect_df = result_df[result_df["dialect_source"] != result_df["standard_source"]]

In [ ]:
print(len(result_df), len(dialect_df))

79872 42669


In [44]:
r = "gangwon"

trans_file_path = os.path.join(path_to_trans_result, "{0}/{1}.json".format(r, m))
with open(trans_file_path, mode="rt", encoding="utf-8") as f:
    json_list = json.loads(f.read())

result_df = pd.DataFrame(json_list)
dialect_df = result_df[result_df["dialect_source"] != result_df["standard_source"]]

In [45]:
print(len(result_df), len(dialect_df))

91008 24084


In [46]:
r = "chungcheong"

trans_file_path = os.path.join(path_to_trans_result, "{0}/{1}.json".format(r, m))
with open(trans_file_path, mode="rt", encoding="utf-8") as f:
    json_list = json.loads(f.read())

result_df = pd.DataFrame(json_list)
dialect_df = result_df[result_df["dialect_source"] != result_df["standard_source"]]

In [47]:
print(len(result_df), len(dialect_df))

94848 15601
